Importar librerias

In [14]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets 
import ast
import gzip
import pandas as pd
from io import StringIO


Crear rutas de archivos

In [20]:
# ruta de archivos JSON
ruta_reviews = (r'C:\Users\computadora\Downloads\Labs Henrys PI\ProyectoIndividualML.JC\DATASET_ORIG\australian_user_reviews.json')

Leer archivo Json reviews

In [21]:
def lac():
    with open(ruta_reviews, "rt", encoding="utf-8") as file:
        return [ast.literal_eval(line.strip()) for line in file]     

datos = lac()
df_user_reviews_orig = pd.DataFrame(datos)


Eliminar espacios en blanco

In [22]:
# Aplicar strip() a cada celda del DataFrame para eliminar espacios en blanco
df_user_reviews_orig = df_user_reviews_orig.applymap(lambda x: x.strip() if isinstance(x, str) else x)

Crear CSV original reviews. 

Asi podemos dejar de utilizar el archivo Json que es mas pesado al momento de realizar su descarga y utilizar uno mas manejable como es CSV

In [25]:
# Ruta donde deseas guardar el archivo CSV
ruta_csv = r"C:\Users\computadora\Downloads\Labs Henrys PI\ProyectoIndividualML.JC\DF_CSV_FUNCIONES\df_reviews_orig.csv"  # Cambia esta ruta a la que desees

# Guarda el DataFrame en un archivo CSV
df_user_reviews_orig.to_csv(ruta_csv, index=False)  # El argumento 'index=False' evita que se incluya el índice en el archivo CSV

print("DataFrame guardado como archivo CSV en:", ruta_csv)

DataFrame guardado como archivo CSV en: C:\Users\computadora\Downloads\Labs Henrys PI\ProyectoIndividualML.JC\DF_CSV_FUNCIONES\df_reviews_orig.csv


utilizamos el metodo info() para visualizar el tipo de datos, la catidad de registros, catidad de columnas y el  tamaño en memoria de la data.

In [29]:
df_user_reviews_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [28]:
df_user_reviews_orig.sample(3)

,user_id,user_url,reviews
10551,Calcipher44,http://steamcommunity.com/id/Calcipher44,"[{'funny': '', 'posted': 'Posted October 2, 20..."
14590,MrExecutionerQQ,http://steamcommunity.com/id/MrExecutionerQQ,"[{'funny': '', 'posted': 'Posted November 19, ..."
18569,zandypel,http://steamcommunity.com/id/zandypel,"[{'funny': '', 'posted': 'Posted June 23, 2014..."


In [31]:
df_user_reviews_orig['reviews'].sample(3)

16000    [{'funny': '', 'posted': 'Posted April 2, 2015...
2348     [{'funny': '1 person found this review funny',...
14684    [{'funny': '', 'posted': 'Posted September 29,...
Name: reviews, dtype: object

Aca visualizamos la data atraves de un Datframe y evidenciamos que posee:

  * 3 columnas 
  * Una contien el user_id que podriamos determinar como la columna de identificacion unica, * Dos la user_url que contiene datos de las url suponemos del usuario ya que al finalizar coinciden con el user_id 
  * y por ultimo una columna de reviews que contine datos en formato json anidados.

Crear DF solo con la columna user_id y reviews para desnaidar los datos que se encuentras dentro de ella

In [8]:
df_user_reviews_orig[['user_id','reviews']].head(3)

,user_id,reviews
0,76561197970982479,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."


Limpieza de Datos al DF

Verificacion de datos duplicados en columna user_id

In [9]:
duplicados = df_user_reviews_orig.duplicated('user_id')
duplicados2 = df_user_reviews_orig[duplicados]
duplicados2

,user_id,user_url,reviews
456,bokkkbokkk,http://steamcommunity.com/id/bokkkbokkk,"[{'funny': '', 'posted': 'Posted September 24,..."
1182,ImSeriouss,http://steamcommunity.com/id/ImSeriouss,"[{'funny': '', 'posted': 'Posted January 10, 2..."
1456,76561198062039159,http://steamcommunity.com/profiles/76561198062...,"[{'funny': '', 'posted': 'Posted August 24, 20..."
1477,76561198045009232,http://steamcommunity.com/profiles/76561198045...,"[{'funny': '', 'posted': 'Posted October 31, 2..."
1746,nitr0ticwolf,http://steamcommunity.com/id/nitr0ticwolf,"[{'funny': '', 'posted': 'Posted December 12, ..."
...,...,...,...
17819,76561198076474887,http://steamcommunity.com/profiles/76561198076...,"[{'funny': '', 'posted': 'Posted April 12.', '..."
17916,yolofaceguy,http://steamcommunity.com/id/yolofaceguy,"[{'funny': '', 'posted': 'Posted October 31, 2..."
18028,76561198075591109,http://steamcommunity.com/profiles/76561198075...,"[{'funny': '', 'posted': 'Posted December 26, ..."
18234,76561198092022514,http://steamcommunity.com/profiles/76561198092...,"[{'funny': '', 'posted': 'Posted July 3.', 'la..."


Eliminar registros duplicados

In [10]:
df_user_reviews_orig = df_user_reviews_orig.drop_duplicates(subset=['user_id'])


Verificra regsitros Nulos

In [11]:
df_user_reviews_orig.isnull().sum()

user_id     0
user_url    0
reviews     0
dtype: int64

Verificar resgitros en blancos

In [12]:
df_user_reviews_orig.isna().sum()

user_id     0
user_url    0
reviews     0
dtype: int64

In [13]:
# Ruta donde deseas guardar el archivo CSV
ruta_csv = r"C:\Users\computadora\Downloads\Labs Henrys PI\df_reviews_orig_limpio.csv"  # Cambia esta ruta a la que desees

# Guarda el DataFrame en un archivo CSV
df_user_reviews_orig.to_csv(ruta_csv, index=False)  # El argumento 'index=False' evita que se incluya el índice en el archivo CSV

print("DataFrame guardado como archivo CSV en:", ruta_csv)

DataFrame guardado como archivo CSV en: C:\Users\computadora\Downloads\Labs Henrys PI\df_reviews_orig_limpio.csv


Desanidar DF datos_reviews

Observar la estructura de la primera fila de los valores

In [14]:
primera_review = df_user_reviews_orig['reviews'].iloc[0]
primera_review

[{'funny': '',
  'posted': 'Posted November 5, 2011.',
  'last_edited': '',
  'item_id': '1250',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
 {'funny': '',
  'posted': 'Posted July 15, 2011.',
  'last_edited': '',
  'item_id': '22200',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': "It's unique and worth a playthrough."},
 {'funny': '',
  'posted': 'Posted April 21, 2011.',
  'last_edited': '',
  'item_id': '43110',
  'helpful': 'No ratings yet',
  'recommend': True,
  'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!'}]

In [15]:
# Inicializa una lista vacía para almacenar los datos de la columna "reviews"
datos_reviews = []

# Recorre el DataFrame y la columna "reviews"
for index, fila in df_user_reviews_orig.iterrows():
    for review in fila["reviews"]:
        datos_reviews.append({
            "user_id": fila["user_id"],
            "user_url": fila["user_url"],
            "funny": review.get("funny", ""),  
            "posted": review.get("posted", ""),
            "last_edited": review.get("last_edited", ""),
            "item_id": review.get("item_id", ""),
            "helpful": review.get("helpful", ""),
            "recommend":review.get("recommend", ""),
            "review":review.get("review", "")
        })

# Convierte la lista de datos en un nuevo DataFrame
datos_reviews_desanidados = pd.DataFrame(datos_reviews)

# Muestra las primeras filas del nuevo DataFrame
datos_reviews_desanidados.head(3)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...


Crear CSV Desanidados reviews

In [16]:
# Ruta donde deseas guardar el archivo CSV
ruta_csv = r"C:\Users\computadora\Downloads\Labs Henrys PI\datos_reviews_desanidados.csv"  # Cambia esta ruta a la que desees

# Guarda el DataFrame en un archivo CSV
datos_reviews_desanidados.to_csv(ruta_csv, index=False)  # El argumento 'index=False' evita que se incluya el índice en el archivo CSV

print("DataFrame guardado como archivo CSV en:", ruta_csv)

DataFrame guardado como archivo CSV en: C:\Users\computadora\Downloads\Labs Henrys PI\datos_reviews_desanidados.csv


In [17]:
datos_reviews_desanidados.head(3)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...


Crear un DF solo con la columna ['posted'] para luego desanidarla

In [18]:
def_posted = pd.DataFrame(datos_reviews_desanidados[['user_id','posted']])
def_posted.head(3)

,user_id,posted
0,76561197970982479,"Posted November 5, 2011."
1,76561197970982479,"Posted July 15, 2011."
2,76561197970982479,"Posted April 21, 2011."


Limpiar DF def_posted creado con nuevo_df_reviews['posted']

In [19]:

# Eliminar los puntos de la columna 'texto' utilizando .str.replace()
def_posted['posted'] = def_posted['posted'].str.replace('.', '')
def_posted['posted'] = def_posted['posted'].str.replace(',', '')
def_posted['posted'] = def_posted['posted'].str.replace('Posted', '')



In [20]:
def_posted.head(3)

,user_id,posted
0,76561197970982479,November 5 2011
1,76561197970982479,July 15 2011
2,76561197970982479,April 21 2011


Desanidar el DF def_posted

In [21]:
primera_fila = def_posted['posted'].iloc[0]
primera_fila

' November 5 2011'

In [22]:
# Dividir la columna 'posted' en nuevas columnas de día, mes y año
def_posted[['month', 'day', 'year']] = def_posted['posted'].str.split(' ', n=2, expand=True)

# Eliminar la columna 'posted' original
def_posted.drop(columns=['posted'], inplace=True)

# Eliminar la columna 'month' original
def_posted.drop(columns=['month'], inplace=True)

In [23]:
def_posted.head(3)

,user_id,day,year
0,76561197970982479,November,5 2011
1,76561197970982479,July,15 2011
2,76561197970982479,April,21 2011


In [24]:
# Usando .rename() para cambiar el nombre de la columna
def_posted = def_posted.rename(columns={'day': 'month'})

In [25]:
def_posted.head(3)

,user_id,month,year
0,76561197970982479,November,5 2011
1,76561197970982479,July,15 2011
2,76561197970982479,April,21 2011


Modificra la columna 'month' de letras a numeros str

In [26]:

# Diccionario de correspondencia entre meses y números
meses_a_numeros = {
    'January': '01',
    'February': '02',
    'March': '03',
    'April': '04',
    'May': '05',
    'June': '06',
    'July': '07',
    'August': '08',
    'September': '09',
    'October': '10',
    'November': '11',
    'December': '12'
}

# Usar .map() para transformar los nombres de los meses en números de dos dígitos
def_posted['meses_numeros'] = def_posted['month'].map(meses_a_numeros)

# Eliminar la columna 'month' original
def_posted.drop(columns=['month'], inplace=True)

In [27]:
def_posted.head(3)

,user_id,year,meses_numeros
0,76561197970982479,5 2011,11
1,76561197970982479,15 2011,07
2,76561197970982479,21 2011,04


Desagrupar la columna 'year'

In [28]:
# Dividir la columna 'year' en nuevas columnas de día y año
def_posted[['day', 'year2']] = def_posted['year'].str.split(' ', n=2, expand=True)

# Eliminar la columna 'year' original
def_posted.drop(columns=['year'], inplace=True)


In [29]:
# Usando .rename() para cambiar el nombre de la columna
def_posted = def_posted.rename(columns={'year2': 'year'})
def_posted = def_posted.rename(columns={'meses_numeros': 'mes'})

In [30]:
# Mostrar el resultado
def_posted.sample(frac=1)

,user_id,mes,day,year
6957,2345254252312312,12,23,2015
18441,76561198070565987,06,28,2014
56067,76561198095217647,07,22,2015
24237,abhinavfire5,12,5,2014
20530,76561198040340682,12,3,2013
...,...,...,...,...
34895,Mofflang,12,19,2014
43079,thabogg,04,6,2015
49490,donni1234,05,24,None
52632,changook,12,30,2013


verificra Nulos en el DF creado

In [31]:
def_posted.isnull().sum()

user_id       0
mes           0
day           0
year       9932
dtype: int64

Imputar valores Nulos conla la moda en year

In [32]:
# ver el valor que mas se repite
def_posted["year"].mode()

# imputar el valor en la columna year
def_posted["year"].fillna(def_posted["year"].mode()[0], inplace=True)

In [33]:
def_posted.isnull().sum()

user_id    0
mes        0
day        0
year       0
dtype: int64

In [34]:
# Mostrar el resultado
def_posted.sample(frac=1)

,user_id,mes,day,year
51932,76561198072660787,08,8,2013
16032,jtwh,06,24,2013
51150,razzer_serbia_2,04,18,2015
33113,76561198140978116,06,30,2014
21730,gamingislife,01,28,2015
...,...,...,...,...
9785,76561198057800009,03,12,2014
38153,neojaeger,12,26,2015
35602,AchievementHunterHazard,10,31,2015
33009,YourNotMyRealDad,10,1,2015


Crear una columna completa con los datos de las columnas def_posted'['mes','day','year']. Asegurando 2 digitos para dia y mes

In [35]:
def_posted['fecha'] = def_posted.apply(lambda row: f"{row['year']}-{str(row['mes']).zfill(2)}-{str(row['day']).zfill(2)}", axis=1)

In [36]:
def_posted

,user_id,mes,day,year,fecha
0,76561197970982479,11,5,2011,2011-11-05
1,76561197970982479,07,15,2011,2011-07-15
2,76561197970982479,04,21,2011,2011-04-21
3,js41637,06,24,2014,2014-06-24
4,js41637,09,8,2013,2013-09-08
...,...,...,...,...,...
58425,76561198312638244,07,10,2014,2014-07-10
58426,76561198312638244,07,8,2014,2014-07-08
58427,LydiaMorley,07,3,2014,2014-07-03
58428,LydiaMorley,07,20,2014,2014-07-20


Cambiar de la columna ['recommend'] los valores True por 1 y los False por 0

In [37]:
datos_reviews_desanidados[['recommend']] = datos_reviews_desanidados[['recommend']].replace({True: 1, False: 0})

# Imprimir el nuevo DataFrame df_recommended
datos_reviews_desanidados[['recommend']].sample(frac=1)

,recommend
20134,1
18396,1
7000,1
379,0
43934,1
...,...
10968,0
53515,1
31993,0
38255,1


Verificar valores nulos

In [38]:
datos_reviews_desanidados[['recommend']].isnull().sum()

recommend    0
dtype: int64

Crear el DF para la API con las columnas : ['fecha'],['user_id'],['recommend']. para funcion 2 solicitada

In [39]:
columnas_seleccionadas = [def_posted['fecha'],datos_reviews_desanidados['user_id'],datos_reviews_desanidados['recommend'] ] 
# Lista de nombres de columnas a seleccionar

# Crear un nuevo DataFrame df_original con las columnas seleccionadas
def_reviews_api2 = pd.concat(columnas_seleccionadas, axis=1)

def_reviews_api2.sample(frac=1)


,fecha,user_id,recommend
16777,2014-02-15,ProxiiZ,1
3352,2013-12-17,kuroTENSHI_bread,1
30614,2014-04-26,DisIsHardCore,1
26175,2014-01-04,76561198050771257,1
29307,2014-01-23,Feytico,1
...,...,...,...
1625,2015-10-17,SSAANNEESS,0
19335,2015-04-17,RevivedShadow,1
19162,2013-12-08,jkjmnb,1
48706,2013-12-27,Busterfool,1


Guardar el DF del API2 para la funcion 2 en un archivo CSV para usarlo posteriormente

In [40]:

# Ruta donde deseas guardar el archivo CSV
ruta_csv = r"C:\Users\computadora\Downloads\Labs Henrys PI\def_reviews_api2.csv"  # Cambia esta ruta a la que desees

# Guarda el DataFrame en un archivo CSV
def_reviews_api2.to_csv(ruta_csv, index=False)  # El argumento 'index=False' evita que se incluya el índice en el archivo CSV

print("DataFrame guardado como archivo CSV en:", ruta_csv) 

DataFrame guardado como archivo CSV en: C:\Users\computadora\Downloads\Labs Henrys PI\def_reviews_api2.csv


Llamar a DF desde el csv creado para la funcion 2 de la API

In [41]:
def_reviews_api2=pd.read_csv(ruta_csv)
def_reviews_api2

,fecha,user_id,recommend
0,2011-11-05,76561197970982479,1
1,2011-07-15,76561197970982479,1
2,2011-04-21,76561197970982479,1
3,2014-06-24,js41637,1
4,2013-09-08,js41637,1
...,...,...,...
58425,2014-07-10,76561198312638244,1
58426,2014-07-08,76561198312638244,1
58427,2014-07-03,LydiaMorley,1
58428,2014-07-20,LydiaMorley,1


In [42]:

# Contar las ocurrencias únicas del usuario (user_id) en la columna 'user_id'
unique_user_count = def_reviews_api2['user_id'].nunique()

# Mostrar el resultado
print(unique_user_count)

25457


def countreviews( YYYY-MM-DD y YYYY-MM-DD : str ): Cantidad de usuarios que realizaron reviews entre las fechas dadas y, el porcentaje de recomendación de los mismos en base a reviews.recommend.

In [43]:
def_reviews_api2

,fecha,user_id,recommend
0,2011-11-05,76561197970982479,1
1,2011-07-15,76561197970982479,1
2,2011-04-21,76561197970982479,1
3,2014-06-24,js41637,1
4,2013-09-08,js41637,1
...,...,...,...
58425,2014-07-10,76561198312638244,1
58426,2014-07-08,76561198312638244,1
58427,2014-07-03,LydiaMorley,1
58428,2014-07-20,LydiaMorley,1


In [44]:


def countreviews(fecha_inicio, fecha_fin):
    # Filtrar el DataFrame por las fechas dadas
    df_filtrado = def_reviews_api2[(def_reviews_api2['fecha'] >= fecha_inicio) & (def_reviews_api2['fecha'] <= fecha_fin)]

    # Calcular la cantidad de usuarios únicos
    usuarios_unicos = df_filtrado['user_id'].nunique()

    # Calcular el porcentaje de recomendación promedio
    porcentaje_promedio = df_filtrado['recommend'].mean() * 100

    # Calcular la cantidad de items
    cantidad_items = len(df_filtrado)

    return usuarios_unicos, porcentaje_promedio, cantidad_items


In [45]:

# Ejemplo de uso


fecha_inicio = '2011-01-01'
fecha_fin = '2014-12-31'

usuarios, porcentaje, cantidad_items = countreviews(fecha_inicio, fecha_fin)

print(f"Cantidad de usuarios que realizaron reviews entre {fecha_inicio} y {fecha_fin}: {usuarios}")
print(f"Porcentaje de recomendación promedio: {porcentaje:.2f} %")
print(f"Cantidad de items: {cantidad_items}")

Cantidad de usuarios que realizaron reviews entre 2011-01-01 y 2014-12-31: 20519
Porcentaje de recomendación promedio: 89.77 %
Cantidad de items: 40210
